In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
import json
import numpy as np

In [97]:
# Codes INSEE des régions métropolitaines
regions = {
    0: "France",
    11: "Île-de-France",
    24: "Centre-Val de Loire",
    27: "Bourgogne-Franche-Comté",
    28: "Normandie",
    32: "Hauts-de-France",
    44: "Grand Est",
    52: "Pays de la Loire",
    53: "Bretagne",
    75: "Nouvelle-Aquitaine",
    76: "Occitanie",
    84: "Auvergne-Rhône-Alpes",
    93: "Provence-Alpes-Côte d’Azur",
    94: "Corse"
}

In [98]:
# Différentes classes d'âge utilisées dans les fichiers de Santé Publique France
code_cl_age90 = {
    0: "tous âges",
    9: "0 à 9 ans",
    19: "10 à 19 ans",
    29: "20 à 29 ans",
    39: "30 à 39 ans",
    49: "40 à 49 ans",
    59: "50 à 59 ans",
    69: "60 à 69 ans",
    79: "70 à 79 ans",
    89: "80 à 89 ans",
    90: "90 ans et plus"
    
}

code_cl_ageA = {
    '0': "tous âges",
    'A': "moins de 15 ans",
    'B': "15-44 ans",
    'C': "45-64 ans",
    'D': "65-74 ans",
    'E': "75 ans et plus"
}

In [99]:
# Correspondance département région - attention le site INSEE connaît des fois des indisponibilités
dep_region = pd.read_csv('https://www.insee.fr/fr/statistiques/fichier/3363419/depts2018-txt.zip',
#dep_region = pd.read_csv('depts2018-txt.zip',
                         sep='\t', encoding='latin1').rename(columns={'DEP': 'dep', 'REGION': 'reg'})[['dep', 'reg']]
dep_region.dep=dep_region.dep.map(lambda x: x.strip())

In [100]:
def anneesemaine(_date):
    _iso = _date.isocalendar()
    _annee = str(_iso[0])
    _jour = str(_iso[1])
    if _iso[1]<10:
        return _annee + "0" + _jour
    return _annee + _jour

def premier_jour_anneesemaine(_anneesemaine):
    return datetime.datetime.strptime(str(_anneesemaine) + '-1', "%G%V-%w").date()

In [101]:
# Deces
deces_cumul = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/08c18e08-6780-452d-9b8c-ae244ad529b3', sep=';')
deces_idf_cumul = deces_cumul.query('reg==11')[['jour', 'cl_age90', 'dc']]

In [102]:
def diff_deces(_deces_idf_cumul, _cl_age90):
    _df = _deces_idf_cumul.query(f'cl_age90=={_cl_age90}').groupby('jour').sum().diff().fillna(0).reset_index()
    _df['cl_age90'] = _cl_age90
    return _df

In [103]:
deces_idf = pd.concat([diff_deces(deces_idf_cumul, cl_age90)
                       for cl_age90 in deces_idf_cumul.cl_age90.unique()])

In [104]:
urgences_sos = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/d2af5160-a21d-47b7-8f30-3c20dade63b1', sep=';')

# Admissions aux urgences
urgences_idf = urgences_sos.query('reg==11')[['date_de_passage', 'sursaud_cl_age_corona',
                                              'nbre_pass_corona', 'nbre_pass_tot']]
urgences_idf.columns = ['jour', 'cl_ageA', 'passages_urgence_covid', 'passages_urgence']

# SOS Médecins
sos_medecins_idf = urgences_sos.query('reg==11')[['date_de_passage', 'sursaud_cl_age_corona',
                                                  'nbre_acte_corona', 'nbre_acte_tot']]

sos_medecins_idf.columns = ['jour', 'cl_ageA', 'actes_sos_covid', 'actes_sos']

In [105]:
# IRAs
_response = requests.get('https://www.sentiweb.fr/datasets/incidence-REG-25.json')
_data = json.loads(_response.text)['data']

ira_sentinelles_idf = pd.DataFrame(
    [(str(d['week']), d['inc_low'], d['inc'], d['inc_up']) for d in _data if d['geo_insee']=='11'])
ira_sentinelles_idf.columns=['semaine_str', 'iras_bas', 'iras_moy', 'iras_haut']

In [106]:
# Lissage des IRAs moyens sur 2 semaines

ira_sentinelles_idf['semaine'] = pd.to_datetime(ira_sentinelles_idf['semaine_str'].map(premier_jour_anneesemaine))

iras_liss = ira_sentinelles_idf.groupby('semaine')['iras_moy'].sum().rolling(3, min_periods=1).mean()
iras_liss = iras_liss.reset_index().rename(columns={'iras_moy': 'iras_lisse'})
iras_liss['semaine'] = pd.to_datetime(iras_liss['semaine'])

ira_sentinelles_idf = ira_sentinelles_idf.merge(iras_liss, on='semaine')

In [107]:
# Tests
tests_ = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/001aca18-df6a-45c8-89e6-f82d689e6c01',
                     sep=';', dtype={'dep': str})
tests_old_ = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b4ea7b4b-b7d1-4885-a099-71852291ff20'
                         , sep=';').merge(dep_region, on='dep')
jour_changement_format = '2020-05-13'

tests_idf = pd.concat([
    tests_old_.query(f'jour < "{jour_changement_format}" and reg==11')[
        ['jour', 'clage_covid', 'nb_test', 'nb_pos']].rename(columns={'clage_covid': 'cl_age'}),
    tests_.query(f'jour >= "{jour_changement_format}" and reg==11')[['jour', 'cl_age90', 'T', 'P']].rename(
        columns={'cl_age90': 'cl_age', 'T': 'nb_test', 'P': 'nb_pos'})
])

In [108]:
# Ensemble des sources au format {Nom: (dataframe, colonne principale, colonne denominateur pour calculer un  taux)}
sources = {
    'Décès hospitaliers': (deces_idf, 'dc', None),
    'Passages aux urgences': (urgences_idf, 'passages_urgence_covid', 'passages_urgence'),
    'Actes SOS Médecins': (sos_medecins_idf, 'actes_sos_covid', 'actes_sos'),
    'Tests positifs': (tests_idf, 'nb_pos', 'nb_test'),
    'Infections respiratoires aigues': (ira_sentinelles_idf, 'iras_lisse', None)
}

In [109]:
# Calcul de la semaine (= la date du premier jour de la semaine)
for df, col, col_ref in sources.values():
    if 'jour' in df.columns:
        df['jour'] = pd.to_datetime(df['jour'])
        df['semaine'] = pd.to_datetime(df['jour'].map(anneesemaine).map(premier_jour_anneesemaine))

In [110]:
# Premier jour de la semaine en cours, pour arrêter les indicateurs à la semaine pleine précédente
semaine_en_cours = pd.to_datetime(premier_jour_anneesemaine(anneesemaine(datetime.datetime.today())))

In [111]:
# IFR issu de https://www.lepoint.fr/sante/covid-19-maladie-mortelle-mais-a-quel-point-05-11-2020-2399541_40.php#
fatality_rate = {
    19: 0.015,
    29: 0.035,
    39: 0.0804,
    49: 0.2021,
    59: 0.4892,
    69: 1.1984,
    79: 2.8161,
    89: 7.0167,
    90: 17.37,
}

In [112]:
# Loi log-normale issue de https://www.cebm.net/covid-19/estimating-the-infection-fatality-ratio-in-england/
log_normal_jour = {
    2: 0.000000006947995734, 3: 0.0000009943909898, 4: 0.00001971596617, 5: 0.0001432401693,
    6: 0.0005779783016, 7: 0.001603883984, 8: 0.003455634494, 9: 0.006224227032,
    10: 0.009829795751, 11: 0.01405605467, 12: 0.01861268174, 13: 0.02319674114,
    14: 0.02753802951, 15: 0.0314247717, 16: 0.03471251885, 17: 0.0373213738,
    18: 0.03922655123, 19: 0.04044616224, 20: 0.04102881393, 21: 0.04104250453,
    22: 0.04056548696, 23: 0.03967925097, 24: 0.03846347553, 25: 0.03699266,
    26: 0.03533409831, 27: 0.03354687242, 28: 0.03168158182, 29: 0.02978057574,
    30: 0.02787850567, 31: 0.02600306072, 32: 0.02417578673, 33: 0.02241292029,
    34: 0.02072619244, 35: 0.01912357457, 36: 0.01760995129, 37: 0.01618771451,
    38: 0.01485727833, 39: 0.01361751862, 40: 0.01246614312, 41: 0.01139999905,
    42: 0.01041532578, 43: 0.009507959743, 44: 0.008673498479, 45: 0.007907430234,
    46: 0.007205234605, 47: 0.006562459279, 48: 0.005974777115, 49: 0.005438027276,
    50: 0.0049482435, 51: 0.00450167214, 52: 0.004094782149, 53: 0.003724268808,
    54: 0.003387052676, 55: 0.00308027495, 56: 0.002801290255, 57: 0.002547657518,
    58: 0.002317129729, 59: 0.002107642896, 60: 0.00191730467, 61: 0.001744382908,
    62: 0.001587294367, 63: 0.001444593725, 64: 0.001314963007, 65: 0.001197201524,
    66: 0.001090216347, 67: 0.0009930133644, 68: 0.0009046889154, 69: 0.0008244220097,
    70: 0.0007514671119, 71: 0.0006851474745, 72: 0.0006248489956, 73: 0.0005700145734,
    74: 0.0005201389278, 75: 0.0004747638611, 76: 0.0004334739258, 77: 0.0003958924716
}

# Transformation en loi semaine
log_normal_semaine = {
    semaine: sum([proba for jour, proba in log_normal_jour.items() if int(jour/7) == semaine])
    for semaine in range(0,12)
}

In [113]:
# Implementation de la distribution des décès
cls_age90 = [19, 29, 39, 49, 59, 69, 79, 89, 90]
def distrib(jour_ou_semaine, nombre, cl_age90, pas='jour'):
    items = log_normal_semaine.items() if pas == 'semaine' else log_normal_jour.items()
    
    return [((jour_ou_semaine - datetime.timedelta(days=(7 if pas=='semaine' else 1)*delai)),
             100*proba*nombre/fatality_rate[cl_age90]) for delai, proba in items]

In [129]:
# Choix si l'étude se fait par semaine ou par jour
pas = 'semaine'
# Carence à enlever parceque les décès du futur ne sont pas encore connus
carence = -9 if pas == 'semaine' else -56

In [130]:
# Somme des décès par jour/semaine et par classe d'âge
deces = {}
for cl_age90 in cls_age90:
    deces[cl_age90] = deces_idf.query(f'cl_age90=={cl_age90}').groupby(pas).dc.sum()

In [131]:
# Calcul du nombre théorique d'infections par casse d'âge
infections = {}
for cl_age90 in cls_age90:
    _series = pd.DataFrame(deces[cl_age90]).reset_index().apply(lambda x: distrib(x[0], x[1], cl_age90, pas), axis=1, result_type='reduce')
    _distribution = pd.DataFrame([(jour_ou_semaine, _nombre)
                                  for _list in _series
                                  for (jour_ou_semaine, _nombre) in _list],
                                 columns=[pas, 'infections'])
    infections[cl_age90] = _distribution.groupby(pas)['infections'].sum()[:carence]

In [132]:
# Aggrégation
infections_idf = pd.DataFrame(
    pd.concat([infections[cl_age90] for cl_age90 in cls_age90], axis=1).sum(axis=1)).reset_index()

In [133]:
# Rajout du nouveau dataframe au dictionnaire des sources
new_col = "Nombre théorique d'infections"
infections_idf.columns = [pas, new_col]
sources[new_col] = (infections_idf, new_col, None)
if pas == 'jour':
    infections_idf['semaine'] = pd.to_datetime(infections_idf['jour'].map(anneesemaine).map(premier_jour_anneesemaine))

In [134]:
# Calcul des jours entre lesquels on a des points sur toutes les séries
jour_min = semaine_en_cours - datetime.timedelta(days=365)
jour_max = semaine_en_cours
for (df, _, _) in sources.values():
    if pas in df.columns:
        jour_min = df[pas].min() if df[pas].min() > jour_min else jour_min
        jour_max = df[pas].max() if df[pas].max() < jour_max else jour_max

def crop_serie(_serie):
    _filtre = (_serie.index >= jour_min) & (_serie.index <= jour_max)
    return _serie[_filtre]

In [135]:
# Aggrégatio par jour/semaine avec lissage par semaine glissante si l'aggrégation est par jour
def agg_serie(df, pas, col, col_ref, crop=True):
    _filtre = (df[pas] >= jour_min) & (df[pas] <= jour_max)
    _serie = df.groupby(pas)[col].sum()
    if col_ref:
        _serie = _serie / df.groupby(pas)[col_ref].sum()
    if pas == 'jour' and col != new_col:
        _serie = _serie.rolling(7, min_periods=2).mean()
    if crop:
      return crop_serie(_serie)
    return _serie

In [136]:
# Variation d'une semaine à l'autre
def increment_wtow(_serie, pas='jour', col=None):
    _shift = 1 if pas == 'semaine' else 7
    return (_serie / _serie.shift(_shift)) - 1

In [137]:
# Calcul de l'évolution semaine à semaine par indicateur puis aggrégation
_dict = {}
for name, (df, col, col_ref) in sources.items():
    if pas in df.columns:
        _serie = agg_serie(df, pas, col, None)         
        _dict[name] = increment_wtow(_serie, pas, col)
        if col_ref:
            _dict[f'Taux {name}'] = increment_wtow(agg_serie(df, pas, col, col_ref), pas)

evolution = pd.DataFrame(_dict)

In [138]:
# Metriques de mesure d'erreur
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [139]:
# Utilitaire pour afficher la courbe
def show(_fig, _title=''):
  _fig.update_layout(
      title=_title,
      font=dict(
          family="Courier New Bold",
          size=30,
      ),
      #legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.4, bgcolor="rgba(0,0,0,0)")
      legend=dict(orientation="h", yanchor="top", y=1.02, xanchor="right", x=0.9, bgcolor="rgba(0,0,0,0)")
      )
  _fig.update_xaxes(tick0=datetime.date(2020, 1, 6))
  _fig.show()

In [140]:
# Calcul de l'erreur entres les séries
fig = go.Figure()
df = evolution.copy()
df = df[df.index > pd.to_datetime('2020-08-01')]
range_pas = 1 if pas == 'semaine' else 14
_list = []
for col in df.columns:
  if col != new_col:
      grid_search = [
          (_multiplier,
           _shift,
           rmse((_multiplier/100)*df[col].shift(_shift), df[new_col])
          )
          for _multiplier in range(100, 101, 1)
          for _shift in range(-range_pas, 0)
      ]
      _index = np.argmin([_tuple[2] for _tuple in grid_search])
      multiplier, shift, error = grid_search[_index]
      _list.append({"Métrique": col,
                    #"Multiplicateur": multiplier/100, "Retard": shift, 
                    "RMSE": error})
      _serie = (multiplier/100)*df[col].shift(shift)
      fig.add_trace(go.Scatter(x=_serie.index, y=_serie, name=col, line=dict(width=4)))
show(fig, 'Variation de semaine en semaine')
pd.DataFrame(_list).sort_values('RMSE')

,Métrique,RMSE
6,Taux Tests positifs,0.147545
2,Taux Passages aux urgences,0.167139
1,Passages aux urgences,0.198023
5,Tests positifs,0.214854
4,Taux Actes SOS Médecins,0.224167
3,Actes SOS Médecins,0.290457
7,Infections respiratoires aigues,0.494743
0,Décès hospitaliers,0.796160


In [141]:
fig = go.Figure()
for col in evolution.columns:
    fig.add_trace(go.Scatter(x=evolution[col].index, y=evolution[col], name=col, line=dict(width=4)))

In [142]:
# Affichage de toutes les courbes d'indicateurs bruts + taux
base = pd.Series({infections_idf[pas].min(): 0, deces_idf[pas].max(): 0})

fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

for name, (df, col, col_ref) in sources.items():
    if pas in df.columns:
        _serie = agg_serie(df, pas, col, None, crop=False)
        fig.add_trace(go.Scatter(x=_serie.index, y=_serie, name=name, line=dict(width=4)))
        if col_ref:
            _taux = agg_serie(df, pas, col, col_ref, crop=False)
            fig.add_trace(go.Scatter(x=_taux.index,
                                     y=_taux,
                                     name='Taux ' + name + ' (échelle de droite)',
                                     line=dict(width=4)),
                          secondary_y=True)
fig.add_trace(go.Scatter(x=base.index, y=base, name='', line_color='white'))
show(fig, f'Indicateur par {pas}')

In [143]:
# Traçage du nombre théorique d'infections par jour si l'étude est faite par jour
if pas == 'jour':
  fig = go.Figure()
  _serie = infections_idf.groupby('jour')["Nombre théorique d'infections"].sum()
  fig.add_trace(go.Scatter(x=_serie.index, y=_serie, line=dict(width=4)))
  show(fig, "Nombre théorique d'infections par jour")